In [4]:
from imports import *
from information_conditions import Information_Conditions
from base_ecopg import *
from helper_functions import *
from simulation_and_results_functions import *

In [49]:
np.set_printoptions(precision=8, suppress=True, linewidth=200)


In [3]:
def add_degraded_state_policies_both_state_and_action(strategy):
        
        strategy_propserous_and_degraded_state = strategy.copy()

        for i in [0, 2, 4, 6]:
            x = np.random.rand()
            strategy_propserous_and_degraded_state.insert(i, x)
        
        return strategy_propserous_and_degraded_state


def add_degraded_state_policies_only_state(strategy):
        
        strategy_propserous_and_degraded_state = strategy.copy()
        x = np.random.rand()
        strategy_propserous_and_degraded_state.insert(0, x)
        
        return strategy_propserous_and_degraded_state



In [58]:
def create_policy_from_strategy(agent_1_strategy, agent_2_strategy):


    agent_1_strategy = [[x, 1-x] for x in agent_1_strategy]
    agent_2_strategy = [[x, 1-x] for x in agent_2_strategy]

    policy = np.array([agent_1_strategy, agent_2_strategy])

    return policy


def epsilon_noise_for_strategy(determinstic_strategy, epsilon):
    #to avoid pure determinstic strategies, we remove e from 0  
    strategy_with_epsilon = [x + epsilon if x == 0 else x - epsilon if x == 1 else x for x in determinstic_strategy]
    return strategy_with_epsilon



def create_policy_with_noise_from_deterministic_strategy(agent_1_strategy, agent_2_strategy, epsilon):

    
    determinstic_policy = create_policy_from_strategy(agent_1_strategy, agent_2_strategy)
    
    
    agent_1_strategy_with_noise = epsilon_noise_for_strategy(agent_1_strategy, epsilon)
    agent_2_strategy_with_noise = epsilon_noise_for_strategy(agent_2_strategy, epsilon)

    noisy_policy = create_policy_from_strategy(agent_1_strategy_with_noise, agent_2_strategy_with_noise)

    return noisy_policy, determinstic_policy


In [2]:
def create_determinstic_strategies_set_for_both_players(mode):
    '''creates deterministic strategy sets for given mode '''

    # E = 0.05
    mae_ecopg_for_evaluating_no_of_states = create_mae_ecopg_for_given_mode_POstratAC(mode)
    number_of_states = mae_ecopg_for_evaluating_no_of_states.Q
    
    if mode == 'none' or mode == 'social':
        determinstic_strategy_itertools = itertools.product([1 , 0], repeat = number_of_states)
        determinisic_strategy_lists = list(list(strat) for strat in determinstic_strategy_itertools)
        all_determinstic_strategy_dictionary_full = {str(strat):strat for strat in determinisic_strategy_lists}

    else:
        number_of_prosperous_states = int(number_of_states/2)
        determinstic_strategy_itertools = itertools.product([1, 0], repeat = number_of_prosperous_states)
        determinisic_strategy_lists = list(list(strat) for strat in determinstic_strategy_itertools)
        all_determinstic_strategy_dictionary_only_prosperous = {str(strat):strat for strat in determinisic_strategy_lists}

        if mode == 'complete':
             all_determinstic_strategy_dictionary_full = {key: add_degraded_state_policies_both_state_and_action(value) for key, value in all_determinstic_strategy_dictionary_only_prosperous.items()}
        elif mode == 'ecological':
             all_determinstic_strategy_dictionary_full = {key: add_degraded_state_policies_only_state(value) for key, value in all_determinstic_strategy_dictionary_only_prosperous.items()}


    strategy_set_p1 = all_determinstic_strategy_dictionary_full
    strategy_set_p2 = all_determinstic_strategy_dictionary_full

    return strategy_set_p1, strategy_set_p2

In [23]:
def create_determinstic_strategies_set_for_both_players_with_epsilon(mode, epsilon):
    '''creates deterministic strategy sets for given mode '''

    mae_ecopg_for_evaluating_no_of_states = create_mae_ecopg_for_given_mode_POstratAC(mode)
    number_of_states = mae_ecopg_for_evaluating_no_of_states.Q
    
    if mode == 'none' or mode == 'social':
        determinstic_strategy_itertools = itertools.product([1 - epsilon , 0 + epsilon], repeat = number_of_states)
        determinisic_strategy_lists = list(list(strat) for strat in determinstic_strategy_itertools)
        all_determinstic_strategy_dictionary_full = {str(strat):strat for strat in determinisic_strategy_lists}

    else:
        number_of_prosperous_states = int(number_of_states/2)
        determinstic_strategy_itertools = itertools.product([1 - epsilon , 0 + epsilon], repeat = number_of_prosperous_states)
        determinisic_strategy_lists = list(list(strat) for strat in determinstic_strategy_itertools)
        all_determinstic_strategy_dictionary_only_prosperous = {str(strat):strat for strat in determinisic_strategy_lists}

        if mode == 'complete':
             all_determinstic_strategy_dictionary_full = {key: add_degraded_state_policies_both_state_and_action(value) for key, value in all_determinstic_strategy_dictionary_only_prosperous.items()}
        elif mode == 'ecological':
             all_determinstic_strategy_dictionary_full = {key: add_degraded_state_policies_only_state(value) for key, value in all_determinstic_strategy_dictionary_only_prosperous.items()}


    strategy_set_p1 = all_determinstic_strategy_dictionary_full
    strategy_set_p2 = all_determinstic_strategy_dictionary_full

    return strategy_set_p1, strategy_set_p2



Qvalue

In [16]:
def check_policy_for_stability_Q_values(determinstic_policy, mode):
    #works only for determentsic strtagues
    mae = create_mae_ecopg_for_given_mode_POstratAC(mode)

    mae.Ps(determinstic_policy)
    Qioa = mae.Qioa(determinstic_policy)
    obsdist = mae.obsdist(determinstic_policy)

    # print(determinstic_policy)

    E = 0
    
    determinstic_policy_eps = np.where(determinstic_policy == 0, E, determinstic_policy)
    determinstic_policy_eps = determinstic_policy_eps/np.sum(determinstic_policy_eps, axis = 0)

    # print(determinstic_policy_eps)

    Qisa = mae.Qisa(determinstic_policy)
    # print(Qioa)
    #for now checking for all possible states even the ones not visited
    #need to remove discard degraded state


    reverse_of_determinstic_policy = 1 - determinstic_policy

    agents = 0
    actions = 1
    states = 2

    # Qi = Qioa[:, :, 0]
    # Qi_reverse = Qioa[:, :, 1]

    
    Qi = jnp.einsum(Qioa, [agents, states, actions],  determinstic_policy, [agents, states, actions], [agents, states])  #Here, we're calculating the average Q value for each agent - across the policy 
    Qi_reverse = jnp.einsum(Qioa, [agents, states, actions],  reverse_of_determinstic_policy, [agents, states, actions], [agents, states]) 

    print("Qioa \n", Qioa)
    print("Qi \n" , Qi)
    print("Qi_reverse \n", Qi_reverse)


    tolerance  = 0
    stability = np.all(Qi >= Qi_reverse - tolerance)

    return stability


E = 0.01
policy = create_policy_from_strategy([0 + E] , [0 + E])

check_policy_for_stability_Q_values(policy, 'none')

Qioa 
 [[[-298.5296 -298.52  ]]

 [[-298.5296 -298.52  ]]]
Qi 
 [[-298.5201]
 [-298.5201]]
Qi_reverse 
 [[-298.5295]
 [-298.5295]]


Array(True, dtype=bool)

In [ ]:
def check_for_stability_all_determinstic_strategies(mode):

    strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players_with_epsilon(mode)

    index = 1
    for i, p1_strategy in enumerate(list(strategy_set_p1.values())):
        for j, p2_strategy in enumerate(list(strategy_set_p2.values())):

            policy = create_policy_from_strategy(p1_strategy, p2_strategy)
            stability = check_policy_for_stability_Q_values(policy, mode)
            
            # print(stability)
            if stability == True:
                print(index, (list(strategy_set_p1.keys()))[i], (list(strategy_set_p2.keys()))[j])
                index = index + 1

In [ ]:
for mode in [ 'none', "ecological", 'social', 'complete']:
    print(f"===  mode: {mode} ===")
    check_for_stability_all_determinstic_strategies(mode)




===  mode: none ===
1 [0.01] [0.01]
===  mode: ecological ===
1 [0.99] [0.99]
2 [0.01] [0.01]
===  mode: social ===
1 [0.01, 0.01, 0.01, 0.01] [0.01, 0.01, 0.01, 0.01]
===  mode: complete ===
1 [0.99, 0.99, 0.99, 0.99] [0.99, 0.99, 0.99, 0.99]
2 [0.99, 0.01, 0.01, 0.99] [0.99, 0.01, 0.01, 0.99]
3 [0.01, 0.01, 0.01, 0.99] [0.01, 0.01, 0.01, 0.99]
4 [0.01, 0.01, 0.01, 0.01] [0.01, 0.01, 0.01, 0.01]



##Q results

===  mode: none ===
1 [0.01] [0.01]
===  mode: ecological ===
1 [0.99] [0.99]
2 [0.01] [0.01]

===  mode: social ===
1 [0.01, 0.01, 0.01, 0.01] [0.01, 0.01, 0.01, 0.01]
===  mode: complete ===
1 [0.99, 0.99, 0.99, 0.99] [0.99, 0.99, 0.99, 0.99]
2 [0.99, 0.01, 0.01, 0.99] [0.99, 0.01, 0.01, 0.99]
3 [0.01, 0.01, 0.01, 0.99] [0.01, 0.01, 0.01, 0.99]
4 [0.01, 0.01, 0.01, 0.01] [0.01, 0.01, 0.01, 0.01]

In [ ]:
strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players('none')

In [ ]:
mae = create_mae_ecopg_for_given_mode_POstratAC('none')
E = 0.0
policy = create_policy_from_strategy([1], [1])
Qioa = mae.Qioa(policy)

reverse_policy = 1 - policy

print(Qioa)
print(policy)
Vio = mae.Vio(policy)
# print(Qioa)

obsdist = mae.obsdist(policy)

# Qioa_2 =  np.random.randint(low = 0, high = 2, size = Qioa.shape)
# print(Qioa_2, "Qioa_2")
# obsdist_2 = np.random.randint(low = 0, high = 2, size = obsdist.shape)
# print(obsdist_2, "obsdist_2")
# print(obsdist)

agents = 0
actions = 1
states = 2

Qi = jnp.einsum(Qioa, [agents, states, actions],  policy, [agents, states, actions], [agents])  #Here, we're calculating the average Q value for each agent - across the policy and obsdist
print("Qi", Qi)

Qi_reverse = jnp.einsum(Qioa, [agents, states, actions],  reverse_policy, [agents, states, actions], [agents])  #Here, we're calculating the average Q value for each agent - across the policy and obsdist
print("Qi_reverse", Qi_reverse)

# avg_value_agent_1, avg_value_agent_2 = jnp.einsum(Vio, [agents, states], obsdist , [agents, states], [agents])
# print(avg_value_agent_1, avg_value_agent_2, "value of agents")


np.all(Qi < Qi_reverse)
# print(Qioa[0,:,:])

[[[50.   51.91]]

 [[50.   51.91]]]
[[[1 0]]

 [[1 0]]]
Qi [50. 50.]
Qi_reverse [51.91 51.91]


Array(True, dtype=bool)

#strategy vs actions

WSLS - [1, 0, 0, 1] 
Q_ioa for each. 

0.999

value functions -> policy (Q and SA)
policy -> actor critic


actor critic also uses Q values

In [ ]:



def process_and_print_nash_equilibria_results_test(nash_equlibria_result, strategy_set_p1, strategy_set_p2, mode, mae):

    number_of_nash_equilbria = len(nash_equlibria_result)
    print("Total number of Nash ", number_of_nash_equilbria)

    '''better readable format of the strategies'''
    strategies_p1_array = np.array(list(strategy_set_p1.values())) #just the list of strategies
    strategies_p2_array = np.array(list(strategy_set_p2.values()))

    for eq in nash_equlibria_result:                       # each equilibrium profile
        strategy_profile_of_each_agent = [[float(p) for _, p in m] for r, m in eq.mixed_strategies()]    #converting mixed strategis to float

        p1_active_strategy = np.array(strategy_profile_of_each_agent[0]).astype(bool) #boolean conversion - for strategy in pure nash probability =  1, for strategy not in pure nash, probability = 0
        p2_active_strategy = np.array(strategy_profile_of_each_agent[1]).astype(bool)
        # float_profile = np.array() # iterate over (strategy, prob

        p1_nash_strategy = (strategies_p1_array[p1_active_strategy])[0]
        p2_nash_strategy =  (strategies_p2_array[p2_active_strategy])[0]

        
        print('P1:' , p1_nash_strategy)   #will only print the strategy found in nash
        print('P2:' , p2_nash_strategy)

        policy_test = create_policy_from_strategy(p1_nash_strategy, p2_nash_strategy)

        print(calculate_avg_value_given_policy_state_test_with_print(policy_test, mode, mae))

        print('------')


def calculate_avg_value_given_policy_state_test_with_print(policy, mode, mae):

    Vis = mae.Vis(policy)
    state_dist = mae.Ps(policy)

    print(Vis, "vis")
    print(state_dist, "state dist")
    
    if mode == 'complete' or mode == 'ecological':
        Vis = Vis[:,1::2]
        state_dist = state_dist[1::2]
        state_dist = state_dist/np.sum(state_dist)
    agents = 0
    states = 1
    avg_value_agent_1, avg_value_agent_2 = jnp.einsum(Vis, [agents, states], state_dist , [states], [agents])

    print(avg_value_agent_1, avg_value_agent_2, "Vi")

    return  avg_value_agent_1, avg_value_agent_2


'''Memo1pd testing for understanding '''



mode = 'social'

env_socdi = SocialDilemma(R=0.8, T=1, S=-0.5, P= 0)
env_memo1pd = HistoryEmbedded(env_socdi, h=(1,1,1))

strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players_with_epsilon(mode)


mae_socdi = stratAC(env=env_memo1pd, learning_rates=0.1, discount_factors= 0.99)

p1_reward_matrix_only_state, p2_reward_matrix_only_state = metagame_reward_matrix_state_test(strategy_set_p1, strategy_set_p2, mode,mae_socdi)

payoffs_row = p1_reward_matrix_only_state
payoffs_col  = p2_reward_matrix_only_state

game = pygambit.Game.from_arrays(payoffs_row, payoffs_col)


result = pygambit.nash.enumpure_solve(game)



process_and_print_nash_equilibria_results_test(result.equilibria, strategy_set_p1, strategy_set_p2, mode, mae_socdi)

Total number of Nash  3
P1: [0.99 0.01 0.01 0.99]
P2: [0.99 0.01 0.01 0.99]
[[77.3666 76.5826 76.5825 77.3665]
 [77.3666 76.5826 76.5825 77.3665]] vis
[0.9607 0.0099 0.0099 0.0195] state dist
77.35103 77.35103 Vi
(Array(77.351, dtype=float32), Array(77.351, dtype=float32))
------
P1: [0.99 0.01 0.01 0.01]
P2: [0.99 0.01 0.01 0.01]
[[27.0722  0.7635  0.7635  0.7635]
 [27.0722  0.7635  0.7635  0.7635]] vis
[0.005  0.0099 0.0099 0.9752] state dist
0.8950479 0.89504796 Vi
(Array(0.895, dtype=float32), Array(0.895, dtype=float32))
------
P1: [0.01 0.01 0.01 0.01]
P2: [0.01 0.01 0.01 0.01]
[[0.503 0.503 0.503 0.503]
 [0.503 0.503 0.503 0.503]] vis
[0.0001 0.0099 0.0099 0.9801] state dist
0.5030022 0.5030023 Vi
(Array(0.503, dtype=float32), Array(0.503, dtype=float32))
------


In [ ]:
'''Memo1pd testing for understanding '''



mode = 'social'

env_socdi = SocialDilemma(R=0.8, T=1, S=-0.5, P= 0)
env_memo1pd = HistoryEmbedded(env_socdi, h=(1,1,1))

strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players_with_epsilon(mode)


mae_socdi = stratAC(env=env_memo1pd, learning_rates=0.1, discount_factors= 0.99)

p1_reward_matrix_only_state, p2_reward_matrix_only_state = metagame_reward_matrix_obs_ecopg(strategy_set_p1, strategy_set_p2, mode,mae_socdi)

payoffs_row = p1_reward_matrix_only_state
payoffs_col  = p2_reward_matrix_only_state

game = pygambit.Game.from_arrays(payoffs_row, payoffs_col)


result = pygambit.nash.enumpure_solve(game)



process_and_print_nash_equilibria_results_test(result.equilibria, strategy_set_p1, strategy_set_p2, mode, mae_socdi)

TypeError: metagame_reward_matrix_obs_ecopg() takes 3 positional arguments but 4 were given

In [53]:
mode = 'social'

env_socdi = SocialDilemma(R=0.8, T=1, S=-0.5, P= 0)
env_memo1pd = HistoryEmbedded(env_socdi, h=(1,1,1))
mae_socdi = stratAC(env=env_memo1pd, learning_rates=0.1, discount_factors= 0.99)


def check_policy_for_stability_Q_values_state(policy):
    agents = 0
    actions = 1
    states = 2

    Qisa = mae_socdi.Qisa(policy)

    reverse_policy = 1 - policy



    Qi = jnp.einsum(Qisa, [agents, states, actions],  policy, [agents, states, actions], [agents])  #Here, we're calculating the average Q value for each agent - across the policy and obsdist

    Qi_reverse = jnp.einsum(Qisa, [agents, states, actions],  reverse_policy, [agents, states, actions], [agents])  #Here, we're calculating the average Q value for each agent - across the policy and obsdist

    # avg_value_agent_1, avg_value_agent_2 = jnp.einsum(Vio, [agents, states], obsdist , [agents, states], [agents])
    # print(avg_value_agent_1, avg_value_agent_2, "value of agents")


    stability =  np.all(Qi < Qi_reverse)
    return stability



def check_policy_for_stability_Q_values_state_all():
    strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players('social')
    strategy_set_p1 = list(strategy_set_p1.values())
    strategy_set_p2 = list(strategy_set_p2.values())

    index = 1
    for i, p1_strategy in enumerate(strategy_set_p1):
        for j, p2_strategy in enumerate(strategy_set_p2):

            policy  = create_policy_from_strategy(p1_strategy, p2_strategy)
            stability = check_policy_for_stability_Q_values_state(policy)
            # print(stability)
            if stability == True:
                print(index, strategy_set_p1[i], strategy_set_p2[j])
                index = index + 1

check_policy_for_stability_Q_values_state_all()
# print(Qioa[0,:,:])    



1 [1, 1, 1, 1] [1, 1, 1, 1]
2 [1, 1, 1, 1] [1, 0, 1, 1]
3 [1, 1, 1, 1] [0, 1, 1, 1]
4 [1, 1, 1, 0] [1, 1, 1, 0]
5 [1, 1, 1, 0] [1, 0, 1, 1]
6 [1, 1, 1, 0] [1, 0, 1, 0]
7 [1, 1, 1, 0] [0, 1, 1, 0]
8 [1, 1, 1, 0] [0, 0, 1, 0]
9 [1, 1, 1, 0] [0, 0, 0, 0]
10 [1, 1, 0, 1] [1, 1, 1, 1]
11 [1, 1, 0, 1] [1, 1, 1, 0]
12 [1, 1, 0, 1] [1, 0, 1, 1]
13 [1, 1, 0, 1] [1, 0, 1, 0]
14 [1, 1, 0, 1] [0, 1, 1, 1]
15 [1, 1, 0, 1] [0, 0, 1, 1]
16 [1, 1, 0, 0] [1, 1, 1, 0]
17 [1, 1, 0, 0] [1, 0, 1, 1]
18 [1, 1, 0, 0] [0, 0, 1, 1]
19 [1, 0, 1, 1] [0, 1, 1, 1]
20 [1, 0, 1, 1] [0, 1, 0, 1]
21 [1, 0, 1, 1] [0, 0, 1, 0]
22 [1, 0, 1, 1] [0, 0, 0, 0]
23 [1, 0, 1, 0] [0, 1, 0, 1]
24 [1, 0, 1, 0] [0, 0, 1, 0]
25 [1, 0, 1, 0] [0, 0, 0, 1]
26 [1, 0, 0, 0] [0, 1, 0, 0]
27 [0, 1, 1, 1] [1, 1, 1, 1]
28 [0, 1, 1, 1] [1, 1, 0, 1]
29 [0, 1, 1, 1] [1, 0, 1, 1]
30 [0, 1, 1, 1] [0, 1, 1, 1]
31 [0, 1, 1, 1] [0, 0, 1, 1]
32 [0, 1, 1, 0] [1, 1, 1, 0]
33 [0, 1, 1, 0] [0, 1, 1, 0]
34 [0, 1, 1, 0] [0, 1, 0, 0]
35 [0, 1, 0, 1] [1, 0, 

Reward prediction error

In [87]:
env_socdi = SocialDilemma(R=0.8, T=1, S=-0.5, P= 0)
env_memo1pd = HistoryEmbedded(env_socdi, h=(1,1,1))
mae_socdi = stratAC(env=env_memo1pd, learning_rates=0.1, discount_factors= 0.99)


def RPE_stability_check(policy):

    prob_of_coop = policy[:, :, 0]
    # print(prob_of_coop)

    reward_prediction_error = mae_socdi.RPEisa(policy)
    # print(reward_prediction_error)
    argmin_Q = np.argmin(reward_prediction_error, axis = 2) #if erro i s -.5 theb the poliviy id 
    # print(argmin_Q)

    stability = np.all(prob_of_coop == argmin_Q)
    
    return stability



def check_policy_for_stability_RPE_all():
    strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players('social')
    strategy_set_p1 = list(strategy_set_p1.values())

    
    strategy_combinations = itertools.combinations_with_replacement(strategy_set_p1,2)
    policies =  np.array([create_policy_from_strategy(p1_strategy, p2_strategy) for p1_strategy, p2_strategy in strategy_combinations])
    results = list(map(RPE_stability_check, policies))
    # print(results)
    # print("==", mode, "===")
    print(policies[results])


check_policy_for_stability_RPE_all()

[[[[1 0]
   [0 1]
   [0 1]
   [1 0]]

  [[1 0]
   [0 1]
   [0 1]
   [1 0]]]


 [[[1 0]
   [0 1]
   [0 1]
   [0 1]]

  [[1 0]
   [0 1]
   [0 1]
   [0 1]]]


 [[[0 1]
   [0 1]
   [0 1]
   [0 1]]

  [[0 1]
   [0 1]
   [0 1]
   [0 1]]]]


In [ ]:
def RPE_stability_check(policy, mae):

    prob_of_coop = policy[:, :, 0]
    # print(prob_of_coop)

    reward_prediction_error = mae.RPEisa(policy)
    # print(reward_prediction_error)
    argmin_Q = np.argmin(reward_prediction_error, axis = 2) #if erro i s -.5 theb the poliviy id 
    # print(argmin_Q)

    stability = np.all(prob_of_coop == argmin_Q)
    
    return stability



def check_policy_for_stability_RPE_all_socdi():
    strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players('social')
    strategy_set_p1 = list(strategy_set_p1.values())

    mae = create_mae_ecopg_for_given_mode_POstratAC(mode)

    strategy_combinations = itertools.combinations_with_replacement(strategy_set_p1,2)
    policies =  np.array([create_policy_from_strategy(p1_strategy, p2_strategy) for p1_strategy, p2_strategy in strategy_combinations])
    results = list(map(RPE_stability_check, policies))
    print(results)
    # print("==", mode, "===")
    print(policies[results])


check_policy_for_stability_RPE_all_socdi()

TypeError: RPE_stability_check() missing 1 required positional argument: 'mae'

In [85]:
def RPE_stability_check(policy, mae, mode):

    prob_of_coop = policy[:, :, 0]  #Take only probability of cooperation
    # print(prob_of_coop)
    
    reward_prediction_error = mae.RPEioa(policy)   
    # print('reward_prediction_error: ', reward_prediction_error)
    argmin_Q = np.argmin(reward_prediction_error, axis = 2)  # argmin 0 implies RPE is for reducing cooperation, 1 implies RPE is for increasing cooperation
    obsdist = mae.obsdist(policy)
    # print("obsdist \n", obsdist)

    if mode == 'complete' or mode == 'ecological':
        prob_of_coop = prob_of_coop[:,1::2]
        argmin_Q = argmin_Q[:, 1::2]

    # print("prob_of_coop \n", prob_of_coop)
    # print("argin_Q \n", argmin_Q)
    print("---")
    # print(reward_prediction_error)
    stability = np.all(prob_of_coop == argmin_Q)  #if for example, for a particular state you get a zero (RPE reduces for cooperation, ) but your policy is 0. So vector is in the direction of what you're doing.
                                                 #If for a particular state argmin index is 1, then it means you should reduce defection, (thus increase cooperation). if your policy is 0 (no cooperation), then the vector is in the oppoosite of what youre doing
    return stability

# mae_mode = create_mae_ecopg_for_given_mode_POstratAC('social')

# policy = create_policy_from_strategy([1,1,1,1], [1,1,1,1])
# RPE_stability_check(policy, mae_mode)

def check_policy_for_stability_RPE_all(mode):
    strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players(mode)
    strategy_set_p1 = list(strategy_set_p1.values())

    mae = create_mae_ecopg_for_given_mode_POstratAC(mode)
    strategy_combinations = itertools.combinations_with_replacement(strategy_set_p1,2)
    policies =  np.array([create_policy_from_strategy(p1_strategy, p2_strategy) for p1_strategy, p2_strategy in strategy_combinations])
    # print(policies[-1])
    RPE_stability_check_mae = partial(RPE_stability_check, mae = mae, mode = mode)
    results = list(map(RPE_stability_check_mae, policies))
    # print(results)
    # print("==", mode, "===")
    print(policies[results])


check_policy_for_stability_RPE_all('social')


# policy = create_policy_from_strategy([0,0,0, 0], [0,0,0,0])
# RPE_stability_check(policy, create_mae_ecopg_for_given_mode_POstratAC('social'), 'social')

---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
---
[]


In [107]:
mae_social_info = create_mae_ecopg_for_given_mode_POstratAC('social')
policy = create_policy_from_strategy([0,0,0, 0.001], [0,0,0,0.001])


result = run_simulation_for_initial_condition_with_traj(
                mae = mae_social_info, 
                initial_condition = policy,
                make_degraded_state_cooperation_probablity_zero_at_end= False,
                make_degraded_state_obsdist_zero_at_end= False
            )  


print(mae_social_info.TDerror(policy))
print(result['xtraj'])

alpha = mae_social_info.alpha
TDe = mae_social_info.TDerror(policy)
print('TDe: \n', TDe)
n = jnp.newaxis
XexpaTDe = policy * jnp.exp(alpha[:,n,n] * TDe)
print('XexpaTDe: \n', XexpaTDe)
X_next = XexpaTDe / XexpaTDe.sum(-1, keepdims=True)



print(mae_social_info.step(policy)[0])


[[[ 0.      0.    ]
  [-0.0096  0.    ]
  [-0.0097  0.    ]
  [ 0.2827 -0.0003]]

 [[ 0.      0.    ]
  [-0.0096  0.    ]
  [-0.0097  0.    ]
  [ 0.2827 -0.0003]]]
[[[[0.    1.   ]
   [0.    1.   ]
   [0.    1.   ]
   [0.001 0.999]]

  [[0.    1.   ]
   [0.    1.   ]
   [0.    1.   ]
   [0.001 0.999]]]


 [[[0.    1.   ]
   [0.    1.   ]
   [0.    1.   ]
   [0.001 0.999]]

  [[0.    1.   ]
   [0.    1.   ]
   [0.    1.   ]
   [0.001 0.999]]]]
TDe: 
 [[[ 0.      0.    ]
  [-0.0096  0.    ]
  [-0.0097  0.    ]
  [ 0.2827 -0.0003]]

 [[ 0.      0.    ]
  [-0.0096  0.    ]
  [-0.0097  0.    ]
  [ 0.2827 -0.0003]]]
XexpaTDe: 
 [[[0.    1.   ]
  [0.    1.   ]
  [0.    1.   ]
  [0.001 0.999]]

 [[0.    1.   ]
  [0.    1.   ]
  [0.    1.   ]
  [0.001 0.999]]]
[[[0.    1.   ]
  [0.    1.   ]
  [0.    1.   ]
  [0.001 0.999]]

 [[0.    1.   ]
  [0.    1.   ]
  [0.    1.   ]
  [0.001 0.999]]]


In [ ]:
### testing this for socdi

env_socdi = SocialDilemma(R=0.8, T=1, S=-0.5, P= 0)
env_memo1pd = HistoryEmbedded(env_socdi, h=(1,1,1))
mae_socdi = stratAC(env=env_memo1pd, learning_rates=0.1, discount_factors= 0.99)



def next_step_stability_check(policy, mae, mode):

    # prob_of_coop = policy[:, :, 0]  #Take only probability of cooperation
    # print(prob_of_coop)
    
    next_step_policy = mae.step(policy)[0] 
    # print('policy: ', policy)
    # print('next_step_policy: ', next_step_policy)
    # print('reward_prediction_error: ', reward_prediction_error)

    # if mode == 'complete' or mode == 'ecological':
    #     prob_of_coop = prob_of_coop[:,1::2]
    #     argmin_Q = argmin_Q[:, 1::2]

    stability = False
    if np.allclose(next_step_policy, policy, rtol = 0, atol = 0):
        stability = True  #if for example, for a particular state you get a zero (RPE reduces for cooperation, ) but your policy is 0. So vector is in the direction of what you're doing.
                        #If for a particular state argmin index is 1, then it means you should reduce defection, (thus increase cooperation). if your policy is 0 (no cooperation), then the vector is in the oppoosite of what youre doing
    return stability

# mae_mode = create_mae_ecopg_for_given_mode_POstratAC('social')
# e = 1e-5
# policy = create_policy_from_strategy([0,0 +e,1,0], [0,0,0,0])
# print(next_step_stability_check(policy, create_mae_ecopg_for_given_mode_POstratAC('social'), 'social'))

def check_policy_for_stability_next_step_all(mode):
    strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players_with_epsilon(mode,0)
    strategy_set_p1 = list(strategy_set_p1.values())

    mae = mae_socdi
    strategy_combinations = itertools.combinations_with_replacement(strategy_set_p1,2)
    policies =  np.array([create_policy_from_strategy(p1_strategy, p2_strategy) for p1_strategy, p2_strategy in strategy_combinations])
    # print(policies[-1])
    next_step_stability_check_mae = partial(next_step_stability_check, mae = mae, mode = mode)
    results = list(map(next_step_stability_check_mae, policies))
    stable_policies = policies[results]
    # print("==", mode, "===")
    print(len(stable_policies))
    print(stable_policies)


# check_policy_for_stability_RPE_all('social')


# policy = create_policy_from_strategy([0,0,0,0], [0,0,0,0])
# next_step_stability_check(policy, create_mae_ecopg_for_given_mode_POstratAC('social'), 'social')

check_policy_for_stability_next_step_all('social')

0
[]


In [51]:
def next_step_stability_check(policy, mae, mode):

    # prob_of_coop = policy[:, :, 0]  #Take only probability of cooperation
    # print(prob_of_coop)
    
    next_step_policy = mae.step(policy)[0] 
    print('policy \n', policy)
    print('next_step_policy \n', next_step_policy)
    # print('reward_prediction_error: ', reward_prediction_error)

    # if mode == 'complete' or mode == 'ecological':
    #     prob_of_coop = prob_of_coop[:,1::2]
    #     argmin_Q = argmin_Q[:, 1::2]

    stability = False
    if np.allclose(next_step_policy, policy, rtol = 1e-5, atol = 1e-8):
        stability = True  #if for example, for a particular state you get a zero (RPE reduces for cooperation, ) but your policy is 0. So vector is in the direction of what you're doing.
                        #If for a particular state argmin index is 1, then it means you should reduce defection, (thus increase cooperation). if your policy is 0 (no cooperation), then the vector is in the oppoosite of what youre doing
    return stability

# mae_mode = create_mae_ecopg_for_given_mode_POstratAC('social')
e = 1e-5
policy = create_policy_from_strategy([0,0 +e,1,0], [0,0,0,0])
print(next_step_stability_check(policy, create_mae_ecopg_for_given_mode_POstratAC('social'), 'social'))

def check_policy_for_stability_next_step_all(mode):
    strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players_with_epsilon(mode, 1e-5)
    strategy_set_p1 = list(strategy_set_p1.values())

    mae = create_mae_ecopg_for_given_mode_POstratAC(mode)
    strategy_combinations = itertools.combinations_with_replacement(strategy_set_p1,2)
    policies =  np.array([create_policy_from_strategy(p1_strategy, p2_strategy) for p1_strategy, p2_strategy in strategy_combinations])
    # print(policies[-1])
    next_step_stability_check_mae = partial(next_step_stability_check, mae = mae, mode = mode)
    results = list(map(next_step_stability_check_mae, policies))
    stable_policies = policies[results]
    # print("==", mode, "===")
    print(len(stable_policies))
    print(stable_policies)


# check_policy_for_stability_RPE_all('social')


# policy = create_policy_from_strategy([0,0,0,0], [0,0,0,0])
# next_step_stability_check(policy, create_mae_ecopg_for_given_mode_POstratAC('social'), 'social')


# check_policy_for_stability_next_step_all('social')

policy 
 [[[0.      1.     ]
  [0.00001 0.99999]
  [1.      0.     ]
  [0.      1.     ]]

 [[0.      1.     ]
  [0.      1.     ]
  [0.      1.     ]
  [0.      1.     ]]]
next_step_policy 
 [[[0.      1.     ]
  [0.00001 0.99999]
  [1.      0.     ]
  [0.      1.     ]]

 [[0.      1.     ]
  [0.      1.     ]
  [0.      1.     ]
  [0.      1.     ]]]
True


In [ ]:
def next_step_stability_check(policy, mae, mode):

    # prob_of_coop = policy[:, :, 0]  #Take only probability of cooperation
    # print(prob_of_coop)
    
    next_step_policy = mae.step(policy)[0] 
    print('policy: ', policy)
    print('next_step_policy: ', next_step_policy)
    # print('reward_prediction_error: ', reward_prediction_error)

    # if mode == 'complete' or mode == 'ecological':
    #     prob_of_coop = prob_of_coop[:,1::2]
    #     argmin_Q = argmin_Q[:, 1::2]

    stability = False
    if np.allclose(next_step_policy, policy, rtol = 1e-5, atol = 1e-8):
        stability = True  #if for example, for a particular state you get a zero (RPE reduces for cooperation, ) but your policy is 0. So vector is in the direction of what you're doing.
                        #If for a particular state argmin index is 1, then it means you should reduce defection, (thus increase cooperation). if your policy is 0 (no cooperation), then the vector is in the oppoosite of what youre doing
    return stability

# mae_mode = create_mae_ecopg_for_given_mode_POstratAC('social')
e = 1e-5
policy = create_policy_from_strategy([0,0 +e,1,0], [0,0,0,0])
print(next_step_stability_check(policy, create_mae_ecopg_for_given_mode_POstratAC('social'), 'social'))

def check_policy_for_stability_next_step_all(mode):
    strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players_with_epsilon(mode, 1e-5)
    strategy_set_p1 = list(strategy_set_p1.values())

    mae = create_mae_ecopg_for_given_mode_POstratAC(mode)
    strategy_combinations = itertools.combinations_with_replacement(strategy_set_p1,2)
    policies =  np.array([create_policy_from_strategy(p1_strategy, p2_strategy) for p1_strategy, p2_strategy in strategy_combinations])
    # print(policies[-1])
    next_step_stability_check_mae = partial(next_step_stability_check, mae = mae, mode = mode)
    results = list(map(next_step_stability_check_mae, policies))
    stable_policies = policies[results]
    # print("==", mode, "===")
    print(len(stable_policies))
    print(stable_policies)


# check_policy_for_stability_RPE_all('social')


# policy = create_policy_from_strategy([0,0,0,0], [0,0,0,0])
# next_step_stability_check(policy, create_mae_ecopg_for_given_mode_POstratAC('social'), 'social')


# check_policy_for_stability_next_step_all('social')

In [66]:
env_socdi = SocialDilemma(R=0.8, T=1, S=-0.5, P= 0)
env_memo1pd = HistoryEmbedded(env_socdi, h=(1,1,1))
mae_socdi = stratAC(env=env_memo1pd, learning_rates=0.1, discount_factors= 0.99)

In [83]:
def next_step_stability_check_norm(noisy_policy, determinstic_policy, mae, mode):

    # prob_of_coop = policy[:, :, 0]  #Take only probability of cooperation
    # print(prob_of_coop)
    # print(noisy_policy)
    next_step_policy = mae.step(noisy_policy)[0] 
    # print('policy: ', policy)
    # print('next_step_policy: ', next_step_policy)
    # print('reward_prediction_error: ', reward_prediction_error)

    # if mode == 'complete' or mode == 'ecological':
    #     prob_of_coop = prob_of_coop[:,1::2]
    #     argmin_Q = argmin_Q[:, 1::2]

    stability = False
    if np.linalg.norm(next_step_policy - determinstic_policy) <  np.linalg.norm(noisy_policy - determinstic_policy):
        stability = True
        
                          #if for example, for a particular state you get a zero (RPE reduces for cooperation, ) but your policy is 0. So vector is in the direction of what you're doing.
                        #If for a particular state argmin index is 1, then it means you should reduce defection, (thus increase cooperation). if your policy is 0 (no cooperation), then the vector is in the oppoosite of what youre doing
    return stability

# mae_mode = create_mae_ecopg_for_given_mode_POstratAC('social')
# epsilon = 1e-4
# noisy_policy, determinstic_policy = create_policy_with_noise_from_deterministic_strategy([0 ,0,0, 0], [0,0,0,0], epsilon)
# print(next_step_stability_check(noisy_policy, determinstic_policy, create_mae_ecopg_for_given_mode_POstratAC('social'), 'social'))

def check_policy_for_stability_next_step_all(mode):
    strategy_set_p1, strategy_set_p2 = create_determinstic_strategies_set_for_both_players(mode)
    strategy_set_p1 = list(strategy_set_p1.values())

    mae = mae_socdi
    strategy_combinations = itertools.combinations_with_replacement(strategy_set_p1,2)
    policies_list =  np.array([create_policy_with_noise_from_deterministic_strategy(p1_strategy, p2_strategy, 1e-3) for p1_strategy, p2_strategy in strategy_combinations])
    stability_list = []
    for policies in policies_list:
        noisy_policy, determinstic_policy = policies
        stability = next_step_stability_check_norm(noisy_policy, determinstic_policy, mae, mode)
        stability_list.append(stability)


    # next_step_stability_check_mae = partial(next_step_stability_check_norm, mae = mae, mode = mode)
    # results = list(map(next_step_stability_check_mae, policies))
    determinstic_policy_list = np.array([determinstic_policy for (noisy_policy, determinstic_policy) in policies_list])
    # print(stability_list)
    stable_policies = determinstic_policy_list[stability_list]
    # print("==", mode, "===")
    print(len(stable_policies))
    # print(stable_policies)


# check_policy_for_stability_RPE_all('social')


# policy = create_policy_from_strategy([0,0,0,0], [0,0,0,0])
# next_step_stability_check(policy, create_mae_ecopg_for_given_mode_POstratAC('social'), 'social')


check_policy_for_stability_next_step_all('social')

33
